# IMDB

We have to breakdown to multiple files for a group of id, or else memory is not enough.

In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [2]:
import imdb as ib
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile
import types
import numpy as np
import csv

In [3]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

# Load data

In [4]:
def get_imdb_columns():
    imdb = IMDb()
    #get a movie by id
    imdb_movie = imdb.get_movie('0325980')
    #access attributes of the movie by dictionary keys
    imdb.update(imdb_movie)
    imdb_info_column = imdb_movie.keys_alias.values() 
    imdb_info_column = list(set(imdb_info_column))
    if 'imdb_id' not in imdb_info_column:
        imdb_info_column.append('imdb_id')
    if 'tmdb_id' not in imdb_info_column:
        imdb_info_column.append('tmdb_id')
    imdb_info_column.sort()
    return(imdb_info_column)


In [5]:
get_imdb_columns()


['airing',
 'akas',
 'amazon reviews',
 'art direction',
 'assistant director',
 'cast',
 'casting director',
 'certificates',
 'cinematographer',
 'color info',
 'costume department',
 'costume designer',
 'countries',
 'cover url',
 'creator',
 'director',
 'distributors',
 'editor',
 'faqs',
 'full-size cover url',
 'genres',
 'guests',
 'imdb_id',
 'languages',
 'make up',
 'merchandising links',
 'misc links',
 'miscellaneous companies',
 'miscellaneous crew',
 'non-original music',
 'number of episodes',
 'number of seasons',
 'original music',
 'parents guide',
 'photo sites',
 'plot',
 'producer',
 'production companies',
 'production manager',
 'rating',
 'runtimes',
 'set decoration',
 'sound clips',
 'sound crew',
 'special effects',
 'special effects companies',
 'stunt performer',
 'tmdb_id',
 'video clips',
 'visual effects',
 'writer']

In [6]:
def get_movie_details(imdb_info_column,imdb_id, tmdb_id):
    movie_details = []
    try:
        imdb = IMDb()
        imdb_movie = imdb.get_movie(imdb_id)
        imdb.update(imdb_movie)
        if (len(imdb_movie.keys()) > 0) :       
            movie_details = []
            for c in imdb_info_column:
                if imdb_movie.has_key(c):
                    info_field = imdb_movie[c]
                    if info_field is not None:
                        if type(info_field) is list:                    
                            if isinstance(info_field[0], ib.Person.Person):
                                info_list = []
                                for item in info_field:
                                    info_list.append(item.getID())
                                movie_details.append(info_list)
                            elif isinstance(info_field[0], ib.Company.Company):
                                info_list = []
                                for item in info_field:
                                    info_list.append(item.getID())
                                movie_details.append(info_list)  
                            else:
                                movie_details.append(info_field)
                        else:
                            movie_details.append(info_field)
                    else:
                            movie_details.append(None)      

                else:
                    if c == "imdb_id":
                        movie_details.append(imdb_id)
                    elif c == "tmdb_id":
                        movie_details.append(tmdb_id)
                    else:
                        movie_details.append(None)  
        else:
            invalid_imdb_ids.append(i)
            #movies.head(5)
        return movie_details
    except Exception:
        return movie_details

In [7]:
latest_id = 300000

In [8]:
imdb_info_column = get_imdb_columns()
imdb_invalid_id_filename = str(dir_data)+'\\drv_imdb_movie_invalid_id.json'

id_start = 280001
id_range = 20000

while id_start < latest_id:
    id_end = id_start + id_range - 1
    tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details_'+str(id_start)+'_'+str(id_end)+'.json'
    imdb_filename = str(dir_data)+'\\drv_imdb_movie_info_'+str(id_start)+'_'+str(id_end)+'.csv'
   
    movies = pd.DataFrame(columns=imdb_info_column)
    invalid_ids =  pd.DataFrame(columns= ['imdb_id','tmdb_id'])
    count = 0
    
    tmdb_movie = pd.read_json(tmdb_filename)
        
    # only load the one with imdb_id
    tmdb_movie = tmdb_movie[tmdb_movie['imdb_id'].notnull()]
    tmdb_ids = tmdb_movie['id'].tolist()
        
    # if already loaded, no need to reload
    if os.path.isfile(imdb_filename):
        movies = pd.read_csv(imdb_filename,index_col=0, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
        loaded_tmdb_ids = movies['tmdb_id'].tolist()
        if len(loaded_tmdb_ids) >0:
            tmdb_ids = [x for x in tmdb_ids if x not in loaded_tmdb_ids]
        imdb_info_column = movies.columns
        movies = pd.DataFrame(columns=imdb_info_column)
    else:
        movies.to_csv(imdb_filename,header =imdb_info_column, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
                   
    # we don't want to waste our effort to load invalid imdb_id
    if os.path.isfile(imdb_invalid_id_filename):
        invalid_imdb_id_df = pd.read_json(imdb_invalid_id_filename)
        invalid_tmdb_ids = invalid_imdb_id_df['tmdb_id'].tolist()
        if len(invalid_tmdb_ids) >0:
            tmdb_ids = [x for x in tmdb_ids if x not in invalid_tmdb_ids]

    for tmdb_id in tmdb_ids:
        imdb_id = tmdb_movie[tmdb_movie['id']==tmdb_id]['imdb_id'].tolist()[0]
        if imdb_id is not None:
            imdb_id = str(imdb_id.replace('tt',''))
        if imdb_id is None or imdb_id =='':
            invalid_ids = invalid_ids.append({'imdb_id': imdb_id, 'tmdb_id': tmdb_id}, ignore_index=True)

        else:
            movie_details = get_movie_details(imdb_info_column,imdb_id, tmdb_id)
            if len(movie_details) > 0 :
                movies.loc[tmdb_id] = movie_details
            else:
                invalid_ids = invalid_ids.append({'imdb_id': imdb_id, 'tmdb_id': tmdb_id}, ignore_index=True)

        count += 1
        if (count % 50 == 0):
            movies.to_csv(imdb_filename,mode = 'a',header=False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
            invalid_ids.to_json(path_or_buf= imdb_invalid_id_filename)
            movies = pd.DataFrame(columns=imdb_info_column)
        
    movies.to_csv(imdb_filename,mode = 'a',header=False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
    invalid_ids.to_json(path_or_buf= imdb_invalid_id_filename)
    id_start = id_start + id_range


In [9]:
# load one file as example
imdb_filename = str(dir_data)+'\\drv_imdb_movie_info_20001_40000.csv'
imdb_movies = pd.read_csv(imdb_filename,index_col=0, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
imdb_movies.head(5)['tmdb_id']

20002.0    20002.0
20003.0    20003.0
20004.0    20004.0
20006.0    20006.0
20007.0    20007.0
Name: tmdb_id, dtype: float64